In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

# EDA

In [2]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [3]:
## drop the columns that are not needed 
df_cleaned=df
df_cleaned=df_cleaned.drop(['url','image_url','county','VIN','size','condition'],axis=1) 

In [4]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
type             92858
paint_color     130203
description         70
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [5]:
# dropping rows due to NAs 

df_cleaned =df_cleaned.dropna(subset=['year','description','fuel','odometer','lat','long','transmission','model'])

In [6]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status      6778
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [7]:
len(df)-len(df_cleaned)

20827

# Filling missing values 

In [8]:
# filling missing values in title status with 'missing' 

df_cleaned['title_status'] = df_cleaned['title_status'].fillna('missing')
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status         0
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [9]:
df_cleaned['transmission'] = df_cleaned['transmission'].fillna('other')

In [10]:
# filling null values from the model. 
from model_extract import process_car_dataset

featured_eng=process_car_dataset(df_cleaned)


Starting data extraction and cleaning...
Data extraction and cleaning completed!


In [11]:
# increase = featured_eng['drive'].value_counts() - df_cleaned['drive'].value_counts()
# print(increase)

In [12]:
# null values reduced 
featured_eng.isnull().sum() - df_cleaned.isnull().sum()

id                  0
region              0
region_url          0
price               0
year                0
manufacturer   -12111
model               0
cylinders      -30367
fuel                0
odometer            0
title_status        0
transmission        0
drive          -71030
type           -40512
paint_color         0
description         0
state               0
lat                 0
long                0
posting_date        0
dtype: int64

In [13]:
featured_eng.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer      3725
model                0
cylinders       138342
fuel                 0
odometer             0
title_status         0
transmission         0
drive            53265
type             48216
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [14]:
# handling the maufacturer column  

# prinitng the values where manufecturer is null 

manufecturer_null = featured_eng[featured_eng['manufacturer'].isna()]

# printing the values where manufecturer is null 

manufecturer_null[['model']].value_counts().to_csv('manufecturer_null.csv')






In [18]:
featured_eng.isna().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer      3725
model                0
cylinders       138342
fuel                 0
odometer             0
title_status         0
transmission         0
drive            53265
type             48216
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64